In [6]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

# Number of tasks to generate
num_tasks = 10
num_slots = 56  # 15-min intervals from 8:00 AM to 10:00 PM

# Random data generation
np.random.seed(42)

# Define time slot categories
morning_slots = list(range(0, 16))  # 8:00 AM to 12:00 PM
afternoon_slots = list(range(16, 32))  # 12:00 PM to 4:00 PM
evening_slots = list(range(32, 56))  # 4:00 PM to 10:00 PM

# Random priorities and difficulties
priority = np.random.randint(1, 6, num_tasks)  # Priority (1-5)
difficulty = np.random.randint(1, 6, num_tasks)  # Difficulty (1-5)

# Random start dates within the next 7 days
start_dates = [datetime.now() + timedelta(days=np.random.randint(0, 8)) for _ in range(num_tasks)]

# Random deadlines within 7 days from the start date
deadlines = [start_date + timedelta(days=np.random.randint(1, 8), hours=np.random.randint(1, 24)) 
             for start_date in start_dates]

# Convert deadlines to slot numbers (relative to current time)
now = datetime.now()
start_of_day = datetime(now.year, now.month, now.day, 8, 0)  # 8:00 AM today
deadline_slots = []

for deadline in deadlines:
    days_diff = (deadline.date() - start_of_day.date()).days
    hours_diff = deadline.hour - 8  # relative to 8 AM
    minutes_diff = deadline.minute
    
    total_slots = days_diff * 56 + (hours_diff * 4) + (minutes_diff // 15)
    # Ensure deadline is within reasonable range for our model
    total_slots = min(total_slots, num_slots - 1)
    total_slots = max(total_slots, 0)
    deadline_slots.append(total_slots)

# Random durations (in blocks of 15 minutes)
durations = np.random.randint(1, 6, num_tasks)  # Duration in blocks of 15 minutes

# Schedule preference mapping
preference_mapping = {
    'Morning': morning_slots,
    'Afternoon': afternoon_slots,
    'Evening': evening_slots
}

# Schedule preference: Randomly assign Morning, Afternoon, Evening
schedule_preference = np.random.choice(['Morning', 'Afternoon', 'Evening'], num_tasks)

# Create a DataFrame to display the generated data
task_data = pd.DataFrame({
    "Task": [f"Task {i+1}" for i in range(num_tasks)],
    "Priority (1-5)": priority,
    "Difficulty (1-5)": difficulty,
    "Start Date": start_dates,
    "Deadline": deadlines,
    "Deadline Slot": deadline_slots,
    "Duration (15-min blocks)": durations,
    "Schedule Preference": schedule_preference
})

# Display the generated data
print(task_data)

# Parameters
stress_threshold = 25
daily_limit = 40  # Max workload in slots

# Commitments: Assuming 0 for free and 15 for occupied time slots
commitments = {j: np.random.choice([0, 1], p=[0.8, 0.2]) * 15 for j in range(num_slots)}

# Initialize model
model = gp.Model()
model.setParam("OutputFlag", 0)

# Decision variables
X = model.addVars(num_tasks, num_slots, vtype=GRB.BINARY, name="X")
L = model.addVars(num_slots, vtype=GRB.CONTINUOUS, lb=0, name="L")

# Weights for objectives
alpha = 1  # Weight for maximizing leisure time
beta = 1   # Weight for minimizing stress

# Objective 1: Maximize Leisure Time
leisure_time_obj = quicksum(L[j] for j in range(num_slots))

# Objective 2: Minimize Stress Level (difficulty * priority)
stress_obj = quicksum(X[i, j] * (difficulty[i] * priority[i]) for i in range(num_tasks) for j in range(num_slots))

# Set the combined objective function (weighted sum)
model.setObjective(alpha * leisure_time_obj - beta * stress_obj, GRB.MAXIMIZE)

# Constraints

# Task assignment - each task must be assigned exactly once
model.addConstrs(
    (quicksum(X[i, j] for j in range(num_slots)) == 1 for i in range(num_tasks)),
    name="TaskAssignment"
)

# Deadline constraints - tasks must be completed before their deadline
for i in range(num_tasks):
    if deadline_slots[i] < num_slots:
        model.addConstr(
            quicksum(X[i, j] * j for j in range(num_slots)) <= deadline_slots[i],
            name=f"Deadline_{i}"
        )

# No task overlap in a slot
model.addConstrs(
    (quicksum(X[i, j] for i in range(num_tasks)) <= 1 for j in range(num_slots)), 
    name="NoOverlap"
)

# Ensure workload limit
model.addConstr(
    quicksum(X[i, j] * durations[i] for i in range(num_tasks) for j in range(num_slots)) <= daily_limit, 
    name="DailyLimit"
)

# Only one difficult task per day
model.addConstr(
    quicksum(X[i, j] for i in range(num_tasks) if difficulty[i] >= 4 for j in range(num_slots)) <= 1, 
    name="HardTaskLimit"
)

# Time balance constraint
model.addConstr(
    quicksum(15 * quicksum(X[i, j] for i in range(num_tasks)) + L[j] for j in range(num_slots)) == 
    quicksum(15 - commitments[j] for j in range(num_slots)), 
    name="TimeBalance"
)

# Schedule preference (morning, afternoon, evening)
for i in range(num_tasks):
    preferred_slots = preference_mapping[schedule_preference[i]]
    for j in range(num_slots):
        if j not in preferred_slots:
            model.addConstr(X[i, j] == 0, name=f"Preference_{i}_{j}")

# Existing commitments (Lock-in school or other fixed commitments)
for j in range(num_slots):
    if commitments[j] > 0:
        model.addConstr(
            quicksum(X[i, j] for i in range(num_tasks)) == 0, 
            name=f"Commitment_{j}"
        )

# Solve model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    print("\nOptimal Schedule Found:\n")
    print(f"{'Task':<10} {'Start Slot':<12} {'Duration (min)':<15} {'Stress Score':<12}")
    print("-" * 50)
    
    for i in range(num_tasks):
        for j in range(num_slots):
            if X[i, j].x > 0.5:
                task_name = f"Task {i+1}"
                stress_score = difficulty[i] * priority[i]
                print(f"{task_name:<10} {j:<12} {durations[i] * 15:<15} {stress_score:<12}")
    
    print("\nTotal Leisure Time:", sum(L[j].x for j in range(num_slots)))
    
    # Print time slot interpretation
    print("\nTime Slot Reference:")
    for j in range(0, num_slots, 4):
        hour = (j // 4) + 8
        am_pm = "AM" if hour < 12 else "PM"
        hour = hour if hour <= 12 else hour - 12
        hour = 12 if hour == 0 else hour
        print(f"Slot {j}: {hour}:00 {am_pm}")
else:
    print("No optimal solution found.")


      Task  Priority (1-5)  Difficulty (1-5)                 Start Date  \
0   Task 1               4                 4 2025-03-29 22:13:06.792420   
1   Task 2               5                 3 2025-04-02 22:13:06.792438   
2   Task 3               3                 5 2025-04-01 22:13:06.792442   
3   Task 4               5                 2 2025-03-31 22:13:06.792445   
4   Task 5               5                 4 2025-03-28 22:13:06.792448   
5   Task 6               2                 2 2025-03-28 22:13:06.792451   
6   Task 7               3                 4 2025-03-30 22:13:06.792454   
7   Task 8               3                 5 2025-03-30 22:13:06.792457   
8   Task 9               3                 1 2025-04-03 22:13:06.792460   
9  Task 10               5                 4 2025-03-29 22:13:06.792463   

                    Deadline  Deadline Slot  Duration (15-min blocks)  \
0 2025-04-03 14:13:06.792420             55                         2   
1 2025-04-10 13:13:06.792438